In [7]:
import onnxruntime as ort
import numpy as np
from PIL import Image
from torchvision import transforms
import time

# Load the ONNX model
model_path = "custom_cnn.onnx"
session = ort.InferenceSession(model_path, providers=["CPUExecutionProvider"])

# Get input and output information
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name
input_shape = session.get_inputs()[0].shape

# Define the preprocessing pipeline
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to 224x224 pixels
    transforms.ToTensor(),  # Convert to Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize
])

# Define the six classes
classes = ["3_long_blade_rotor", "3_short_blade_rotor", "Bird", "Bird+mini-helicopter", "drone", "rc_plane"]

# Load an image (replace with the path to your image)
image_path = "D:/Micro-Classify/ml_model/notebooks/DIAT-uSAT_dataset/3_long_blade_rotor/figure1.jpg"
image = Image.open(image_path)

# Apply the transformations
image = Image.open(image_path)
input_tensor = transform(image)

# Add a batch dimension
input_tensor = input_tensor.unsqueeze(0)

# Convert to NumPy array with correct data type
input_array = input_tensor.numpy().astype(np.float32)


# Start timer for inference
start_time = time.time()

# Run the inference
output = session.run([output_name], {input_name: input_array})

# End timer for inference
end_time = time.time()

# Calculate inference time
inference_time = end_time - start_time

# Process the output
output_probabilities = np.squeeze(output[0])  # Remove unnecessary dimensions
predicted_class = np.argmax(output_probabilities)  # Get index of the highest probability

# Display results
print(f"Predicted Class: {classes[predicted_class]}")
print(f"Probability: {output_probabilities[predicted_class]:.4f}")
print(f"Inference Time: {inference_time:.4f} seconds")

Predicted Class: 3_long_blade_rotor
Probability: 1.1630
Inference Time: 0.0039 seconds


In [13]:
import onnxruntime as ort
import numpy as np
from PIL import Image
from torchvision import transforms
import os
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

# Load the ONNX model
model_path = "custom_cnn.onnx"
session = ort.InferenceSession(model_path, providers=["CPUExecutionProvider"])

# Get input and output information
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

# Define the preprocessing pipeline
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to 224x224 pixels
    transforms.ToTensor(),  # Convert to Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize
])

# Define the six classes
classes = ["3_long_blade_rotor", "3_short_blade_rotor", "Bird", "Bird+mini-helicopter", "drone", "rc_plane"]

def preprocess_image(image_path):
    """
    Preprocess an image for model inference
    """
    image = Image.open(image_path)
    input_tensor = transform(image)
    input_tensor = input_tensor.unsqueeze(0)
    return input_tensor.numpy().astype(np.float32)

def predict_image(session, input_array, input_name, output_name):
    """
    Run inference on a single image
    """
    output = session.run([output_name], {input_name: input_array})
    output_probabilities = np.squeeze(output[0])
    return np.argmax(output_probabilities)

def evaluate_model(dataset_root):
    """
    Evaluate the ONNX model and generate classification report
    
    :param dataset_root: Root directory containing subdirectories for each class
    :return: Tuple of (true labels, predicted labels)
    """
    true_labels = []
    predicted_labels = []
    
    # Iterate through each class directory
    for class_idx, class_name in enumerate(classes):
        class_path = os.path.join(dataset_root, class_name)
        
        # Check if directory exists
        if not os.path.isdir(class_path):
            print(f"Warning: Directory {class_path} does not exist.")
            continue
        
        # Process each image in the class directory
        for image_filename in os.listdir(class_path):
            image_path = os.path.join(class_path, image_filename)
            
            try:
                # Preprocess the image
                input_array = preprocess_image(image_path)
                
                # Predict the class
                predicted_class = predict_image(session, input_array, input_name, output_name)
                
                # Store true and predicted labels
                true_labels.append(class_idx)
                predicted_labels.append(predicted_class)
            
            except Exception as e:
                print(f"Error processing {image_path}: {e}")
    
    return true_labels, predicted_labels

def generate_classification_report(dataset_root):
    """
    Generate and print classification report
    
    :param dataset_root: Root directory containing subdirectories for each class
    """
    # Evaluate the model
    true_labels, predicted_labels = evaluate_model(dataset_root)
    
    # Generate classification report
    report = classification_report(
        true_labels, 
        predicted_labels, 
        target_names=classes, 
        digits=4
    )
    print("Classification Report:")
    print(report)
    
    # Generate confusion matrix
    cm = confusion_matrix(true_labels, predicted_labels)
    plt.figure(figsize=(10,8))
    
    # Create confusion matrix with Matplotlib
    im = plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.colorbar(im)
    
    # Add text annotations
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], 'd'),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
    
    plt.tight_layout()
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.xticks(range(len(classes)), classes, rotation=45)
    plt.yticks(range(len(classes)), classes)
    plt.savefig('confusion_matrix.png')
    plt.close()

# Usage
# Replace with the path to your test dataset root directory
dataset_root = "D:/Micro-Classify/ml_model/notebooks/DIAT-uSAT_dataset"
generate_classification_report(dataset_root)

Classification Report:
                      precision    recall  f1-score   support

  3_long_blade_rotor     0.9834    0.9625    0.9728       799
 3_short_blade_rotor     0.9604    0.9712    0.9658       800
                Bird     1.0000    1.0000    1.0000       800
Bird+mini-helicopter     0.9975    0.9730    0.9851       815
               drone     1.0000    1.0000    1.0000       835
            rc_plane     0.9626    0.9962    0.9791       800

            accuracy                         0.9839      4849
           macro avg     0.9840    0.9838    0.9838      4849
        weighted avg     0.9841    0.9839    0.9839      4849

